In [2]:
import pandas as pd

def FonctionImportDonnees(chemin):
    """
        importe les données du fichier excel, après expertise
    """
    donnees=pd.ExcelFile(chemin).parse(1) #Sélectionne la première feuille
    donnees=donnees.drop(index=[960,952]) #Supprime les lignes avec trop de NA
    donnees=donnees.drop('5°C T50 (h)',axis=1).drop('5°C T50 (j)',axis=1).drop('5°C TMG (h)',axis=1) #Supprime les colonnes avec information identique, ou avec trop de NA
    
    liste=[]
    col={}
    j=0
    
    for i in range(10,16): #Expertise: création des données de croissance
        a=donnees.columns[i]
        b=donnees.columns[i+1]
        liste.append(donnees[b]-donnees[a])
        c='v'+a+'-'+b
        col[j]=c.replace(' ','')
        j=j+1
    croissance=pd.DataFrame(liste).T
    croissance=croissance.rename(col, axis='columns')
    donnees=pd.concat([donnees,croissance],axis=1).dropna()
    donnees.set_index('rep')
    
    donnees=donnees[['Bancs', 'Pop', 'Echantillon', 'rep', 'N° rep', 'camera', 'semis','zone', '5°C TMG (j)', 'Aire sous la courbe', 'v15j-16j', 'v16j-17j', 'v17j-18j','v18j-19j', 'v19j-20j', 'v20j-21j']]
    
    quanti=['5°C TMG (j)', 'Aire sous la courbe', 'v15j-16j', 'v16j-17j', 'v17j-18j','v18j-19j', 'v19j-20j', 'v20j-21j']
    quali=['Bancs', 'Pop', 'N° rep', 'camera', 'semis','zone']
    
    return donnees,quanti,quali

In [6]:
donnees,quanti,quali=FonctionImportDonnees("https://raw.githubusercontent.com/etudiantdatascience/TER-reduction-dimension/master/Data/semis.xlsx")
for variableQuali in quali:
    valeur=donnees[variableQuali].drop_duplicates().values
    donnees[variableQuali]=donnees[variableQuali].replace(valeur,list(range(len(valeur))))

features, labels = donnees[quanti],donnees['zone']

In [9]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from skrebate import ReliefF
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

clf = make_pipeline(ReliefF(n_features_to_select=2, n_neighbors=100),
                    RandomForestClassifier(n_estimators=100))


In [15]:
genetic_data = pd.read_csv('https://github.com/EpistasisLab/scikit-rebate/raw/master/data/'
                           'GAMETES_Epistasis_2-Way_20atts_0.4H_EDM-1_1.tsv.gz',
                           sep='\t', compression='gzip')

features, labels = genetic_data.drop('class', axis=1).values, genetic_data['class'].values

clf = make_pipeline(ReliefF(n_features_to_select=2, n_neighbors=100),
                    RandomForestClassifier(n_estimators=100))

print(np.mean(cross_val_score(clf, features, labels)))

0.7931251349347151


In [16]:
import ReliefF as ok

In [19]:
help(ok.ReliefF)

Help on class ReliefF in module ReliefF.ReliefF:

class ReliefF(builtins.object)
 |  ReliefF(n_neighbors=100, n_features_to_keep=10)
 |  
 |  Feature selection using data-mined expert knowledge.
 |  
 |  Based on the ReliefF algorithm as introduced in:
 |  
 |  Kononenko, Igor et al. Overcoming the myopia of inductive learning
 |  algorithms with RELIEFF (1997), Applied Intelligence, 7(1), p39-55
 |  
 |  Methods defined here:
 |  
 |  __init__(self, n_neighbors=100, n_features_to_keep=10)
 |      Sets up ReliefF to perform feature selection.
 |      
 |      Parameters
 |      ----------
 |      n_neighbors: int (default: 100)
 |          The number of neighbors to consider when assigning feature
 |          importance scores.
 |          More neighbors results in more accurate scores, but takes longer.
 |      
 |      Returns
 |      -------
 |      None
 |  
 |  fit(self, X, y)
 |      Computes the feature importance scores from the training data.
 |      
 |      Parameters
 |    